In [ ]:
import time
import json
import ast
import os
import datetime
import io
from collections import defaultdict

import imageio
import boto3
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
from timezonefinder import TimezoneFinderL
import pytz
from brtdevkit.data import Dataset
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset, imageids_to_dataset_fast

from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
athena = AthenaClient()
s3 = boto3.resource('s3')


In [ ]:
interesting_ids = '64ee6282325fbc2bb41d83e4','64d55c4aa12f47d52a8ab5eb'

In [ ]:
query = f"""
SELECT
    id, camera_location, collected_on, group_id, robot_name
FROM image_jupiter
WHERE
    image_jupiter.id IN {interesting_ids}
LIMIT 1000
"""
print(query)
athena_df = athena.get_df(query)

In [ ]:
row = athena_df.iloc[0]
athena_df

In [ ]:
query = f"""
SELECT
    id, camera_location, collected_on, group_id, robot_name
FROM image_jupiter
WHERE
    camera_location = 'T08' AND
    robot_name = 'halohitchhiker_102'
LIMIT 10
"""
athena.get_df(query)

In [ ]:
def same_robot_names(robot_name):
    subname = robot_name[:-1]
    return subname + '1', subname + '2', subname + '3'

In [ ]:
dfs = []
for _, row in athena_df.iterrows():
    start_t = str(row["collected_on"] - datetime.timedelta(seconds=1.5)).split('.')[0]
    end_t = str(row["collected_on"] + datetime.timedelta(seconds=1.5)).split('.')[0]
    query = f"""
    SELECT
        id, camera_location, collected_on, group_id, robot_name
    FROM image_jupiter
    WHERE
        collected_on > TIMESTAMP'{start_t}' AND
        collected_on < TIMESTAMP'{end_t}' AND
        robot_name IN {same_robot_names(row["robot_name"])}
    LIMIT 1000
    """
    print(query)
    dfs.append(athena.get_df(query))


In [ ]:
print(len(dfs[0]))
dfs[0].groupby('group_id').count()

In [ ]:
both_seq = pd.concat([dfs[0].sort_values('collected_on'), dfs[1].sort_values('collected_on')], axis=0)

In [ ]:
ids = list(dfs[0]['id']) + list(dfs[1]['id'])

In [ ]:
print(len(dfs[0]['id']))
print(len(dfs[1]['id']))
print(len(ids))
print(','.join(ids))

In [ ]:
from aletheia_dataset_creator.config import dataset_config
import itertools
halo_all_cams = []
for k, v in itertools.chain(dataset_config.HALO_CENTER_CAMERA_PAIRS.items(), dataset_config.HALO_NON_CENTER_CAMERA_PAIRS.items()):
    halo_all_cams.append(k)
    halo_all_cams.append(v)
halo_all_cams
new_dset_name = 'halo_all_cam_images_3_seconds_2_sequences'

In [ ]:
imageids_to_dataset_fast(both_seq, both_seq, dataset_name=new_dset_name, dataset_description="2 different 3 second image sequences with ALL camera data (left+right, front+rear+implement pod).")

In [ ]:
dset = Dataset.retrieve(name=new_dset_name)
dset.download(f"{os.environ['DATASET_PATH']}/{new_dset_name}")